In [1]:
import xarray as xr
import pandas as pd
import re
import numpy as np
from math import radians, cos

## Ajuste de arquivo

Os dados dados que a gente vai trablhar vem inicialmente em um arquivo NetCDF que é um arquivo vetorial. 

As informaçlções da cidades pernambucnas vem em um arquivo csv com as cordenas em formato de string DMS (ex.: -07 graus 54 minutos 42 segundos), então faço a conversão dessa string para um numero decimal. Depois faço um ajuste de posição geográfica, pois nossos dados de pernambuco vão de -180° a 180° e no banco de dados da precipitação eles vão de  0° a 360°.

In [2]:
ds = xr.open_dataset('src\precip_mon_mean.nc')
cidades_df = pd.read_csv('src\Posição geográfica das sedes dos municípios.csv')

def converter_coordenada(coord_str):
    numeros = re.findall(r'-?\d+', str(coord_str))
    if len(numeros) < 3:
        raise ValueError(f"Formato inválido de coordenada: {coord_str}")
    graus, minutos, segundos = map(int, numeros[:3])
    sinal = -1 if graus < 0 else 1
    return sinal * (abs(graus) + minutos / 60 + segundos / 3600)

cidades_df['lat_decimal'] = cidades_df['Latitude'].apply(converter_coordenada)
cidades_df['lon_decimal'] = cidades_df['Longitude'].apply(converter_coordenada)
cidades_df['lon_ajustado'] = cidades_df['lon_decimal'].apply(lambda x: x % 360)

Eu extrai a série temporal de precipitação para cada cidade a partir dos dados de grade do NetCDF. O código percorre cada linha do DataFrame cidades_df, que contém as informações geográficas das cidades já com as coordenadas convertidas para o formato decimal e ajustadas. Aí vai ocorrer a seleção de dados de precipitação quando os ponte de latitude e longitude mais próximo. Converto os dados para DataFrame do pandas, pois facilita para mim.

In [3]:
registros = []

for idx, cidade in cidades_df.iterrows():
    dados = ds['precip'].sel(
        lat=cidade['lat_decimal'],
        lon=cidade['lon_ajustado'],
        method='nearest'
    )
    
    df_temp = dados.to_dataframe().reset_index()
    df_temp['Cidade'] = cidade['Cidade']
    df_temp['Latitude'] = cidade['lat_decimal']
    df_temp['Longitude'] = cidade['lon_decimal']
    
    registros.append(df_temp[['Cidade', 'time', 'precip', 'Latitude', 'Longitude']])
    print(f"Processado: {cidade['Cidade']} ({len(df_temp)} registros)")

Processado: Abreu e Lima (560 registros)
Processado: Afogados da Ingazeira (560 registros)
Processado: Afrânio (560 registros)
Processado: Agrestina (560 registros)
Processado: Água Preta (560 registros)
Processado: Águas Belas (560 registros)
Processado: Alagoinha (560 registros)
Processado: Aliança (560 registros)
Processado: Altinho (560 registros)
Processado: Amaraji (560 registros)
Processado: Angelim (560 registros)
Processado: Araçoiaba (560 registros)
Processado: Araripina (560 registros)
Processado: Arcoverde (560 registros)
Processado: Barra de Guabiraba (560 registros)
Processado: Barreiros (560 registros)
Processado: Belém de Maria (560 registros)
Processado: Belém do São Francisco (560 registros)
Processado: Belo Jardim (560 registros)
Processado: Betânia (560 registros)
Processado: Bezerros (560 registros)
Processado: Bodocó (560 registros)
Processado: Bom Conselho (560 registros)
Processado: Bom Jardim (560 registros)
Processado: Bonito (560 registros)
Processado: Brejão

Estas duas linhas realizam a consolidação final dos dados processados e a padronização dos nomes das colunas para facilitar análises futuras.

In [4]:
df_final = pd.concat(registros, ignore_index=True)
df_final.rename(columns={'time': 'Data', 'precip': 'Precipitacao_mm_mes'}, inplace=True)

Salvo os dados

In [5]:
df_final.to_csv('src\precipitacao_cidades_pernambuco_temporal.csv', index=False, encoding='utf-8')

## Analisando os dados de Pernambuco

In [6]:
dados_pe = pd.read_csv('src\precipitacao_cidades_pernambuco_temporal.csv')

Transformo o formato de datas que está em string no formato date

In [7]:
dados_pe['Data'] = pd.to_datetime(dados_pe['Data'])

Este bloco de código realiza uma verificação da integridade temporal dos dados de precipitação para uma cidade específica, identificando se existem meses faltantes na série histórica.

In [8]:
cidade = "Abreu e Lima"
df_cidade = dados_pe[dados_pe['Cidade'] == cidade].copy()

inicio = df_cidade['Data'].min()
fim = df_cidade['Data'].max()
datas_esperadas = pd.date_range(start=inicio, end=fim, freq='MS') 

datas_existentes = pd.to_datetime(df_cidade['Data'].unique())
datas_faltando = datas_esperadas.difference(datas_existentes)

print(f"esperado: {len(datas_esperadas)} meses ({inicio.year}–{fim.year})")
print(f"encontrado: {len(df_cidade)} meses")
print(f"Faltando {len(datas_faltando)} meses:")

esperado: 560 meses (1979–2025)
encontrado: 560 meses
Faltando 0 meses:


Verifico a média geral de todos os municipios

In [9]:
dados_pe.groupby('Cidade')['Precipitacao_mm_mes'].mean()

Cidade
Abreu e Lima              3.057414
Afogados da Ingazeira     3.053409
Afrânio                   1.804672
Agrestina                 3.337996
Alagoinha                 3.337996
                            ...   
Vicência                  3.337996
Vitória de Santo Antão    3.337996
Xexéu                     3.337996
Água Preta                3.337996
Águas Belas               3.337996
Name: Precipitacao_mm_mes, Length: 185, dtype: float64

Verifico a variância de todos os municipios

In [10]:
dados_pe.groupby('Cidade')['Precipitacao_mm_mes'].var()

Cidade
Abreu e Lima              10.116097
Afogados da Ingazeira      8.561663
Afrânio                    4.039630
Agrestina                 12.128383
Alagoinha                 12.128383
                            ...    
Vicência                  12.128383
Vitória de Santo Antão    12.128383
Xexéu                     12.128383
Água Preta                12.128383
Águas Belas               12.128383
Name: Precipitacao_mm_mes, Length: 185, dtype: float64

verifico o descio-padrão de todos os municipios

In [11]:
dados_pe.groupby('Cidade')['Precipitacao_mm_mes'].std()

Cidade
Abreu e Lima              3.180581
Afogados da Ingazeira     2.926032
Afrânio                   2.009883
Agrestina                 3.482583
Alagoinha                 3.482583
                            ...   
Vicência                  3.482583
Vitória de Santo Antão    3.482583
Xexéu                     3.482583
Água Preta                3.482583
Águas Belas               3.482583
Name: Precipitacao_mm_mes, Length: 185, dtype: float64

## Analise descritiva por Mesorregião

In [16]:
mesorregioes = pd.read_csv('src\mesorregios.csv')
dados_completos = pd.merge(dados_pe, mesorregioes, on='Cidade', how='left')

média por mesorregião 

In [17]:
dados_completos.groupby('Mesorregião')['Precipitacao_mm_mes'].mean()

Mesorregião
Agreste Pernambucano          3.337996
Mata Pernambucana             3.252217
Metropolitana do Recife       3.153292
Sertão Pernambucano           2.806245
São Francisco Pernambucano    2.720412
Name: Precipitacao_mm_mes, dtype: float64

Variância por mesorregião

In [18]:
dados_completos.groupby('Mesorregião')['Precipitacao_mm_mes'].var()

Mesorregião
Agreste Pernambucano          12.107030
Mata Pernambucana             11.737468
Metropolitana do Recife       10.581341
Sertão Pernambucano            8.188738
São Francisco Pernambucano     7.648501
Name: Precipitacao_mm_mes, dtype: float64

Descio padrão por mesorregião

In [19]:
dados_completos.groupby('Mesorregião')['Precipitacao_mm_mes'].std()

Mesorregião
Agreste Pernambucano          3.479516
Mata Pernambucana             3.425999
Metropolitana do Recife       3.252897
Sertão Pernambucano           2.861597
São Francisco Pernambucano    2.765592
Name: Precipitacao_mm_mes, dtype: float64

salva os dados em um arquivo

In [20]:
dados_completos.to_csv('src\dados_completos.csv', index=False, encoding='utf-8')